## Dependencies

In [12]:
# Cell 1
import os, json, math
from glob import glob
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
print("TF", tf.__version__, "NumPy", np.__version__)

TF 2.20.0 NumPy 2.2.6


## Config

In [13]:
# Cell 2 - Edit paths here
DATA_ROOT = "../tools/dataset/"
BATCH = 16
EPOCHS = 40
MAX_SEQ_OVERRIDE = None   # set integer to force, else computed 95th percentile
REP_SAMPLES = 200
TFLITE_OUT = "model_int8.tflite"
HEADER_OUT = "model_data.h"
STATS_OUT = "norm_stats.json"
PREPROCESS_OUT = "preprocess.h"
THERM_H, THERM_W, THERM_C = 8, 8, 3
RADAR_DIM = 12
SEED = 42
np.random.seed(SEED); tf.random.set_seed(SEED)
print("Config set. DATA_ROOT =", DATA_ROOT)


Config set. DATA_ROOT = ../tools/dataset/


## Data reading helpers

In [ ]:
# Cell 3
def read_session(folder):
    frames=[]
    files = sorted([p for p in os.listdir(folder) if p.endswith(".jsonl")])
    for fname in files:
        with open(os.path.join(folder, fname),'r') as fh:
            for line in fh:
                d=json.loads(line)
                left=np.array(d["thermal"]["left"]).reshape(8,8).astype(np.float32)
                center=np.array(d["thermal"]["center"]).reshape(8,8).astype(np.float32)
                right=np.array(d["thermal"]["right"]).reshape(8,8).astype(np.float32)
                thermal=np.stack([left,center,right],axis=-1) # H,W,C

                r1=d["mmWave"]["R1"]; r2=d["mmWave"]["R2"]
                radar=np.array([r1["numTargets"], r1["range"], r1["speed"], r1["energy"], float(r1["valid"]),
                                r2["numTargets"], r2["range"], r2["speed"], r2["energy"], float(r2["valid"])], dtype=np.float32)
                mic = np.array([d["mic"]["left"], d["mic"]["right"]], dtype=np.float32)
                radar_mic = np.concatenate([radar, mic]) # (12,)
                frames.append((thermal, radar_mic))
    return frames

def collect_sessions(root):
    sessions, labels, lengths = [], [], []
    for sub in sorted(os.listdir(root)):
        p=os.path.join(root, sub)
        if not os.path.isdir(p): continue
        lab = 1 if "animal" in sub.lower() else 0
        sess = read_session(p)
        if len(sess)==0: continue
        sessions.append(sess); labels.append(lab); lengths.append(len(sess))
    return sessions, labels, lengths

sessions, labels, lengths = collect_sessions(DATA_ROOT)
print(f"Found {len(sessions)} sessions. length stats min {min(lengths)}, max {max(lengths)}, mean {np.mean(lengths):.1f}")


In [4]:
def collect_sessions(root_dir: str):
    sessions, labels, lengths = [], [], []
    for sub in sorted(os.listdir(root_dir)):
        p = os.path.join(root_dir, sub)
        if not os.path.isdir(p):
            continue
        lab = 1 if "animal" in sub.lower() else 0
        sess = read_session(p)
        if len(sess) == 0:
            continue
        sessions.append(sess); labels.append(lab); lengths.append(len(sess))
    return sessions, labels, lengths

In [5]:
# Cell 4: Compute normalization stats and pick MAX_SEQ_LEN
def compute_stats(sessions):
    t_all = []
    r_all = []
    for sess in sessions:
        for t, r in sess:
            t_all.append(t.reshape(-1))
            r_all.append(r)
    t_all = np.concatenate(t_all).astype(np.float32)
    r_all = np.concatenate(r_all).astype(np.float32)
    return {
        "thermal_mean": float(np.mean(t_all)),
        "thermal_std": float(np.std(t_all) + 1e-6),
        "radar_mean": float(np.mean(r_all)),
        "radar_std": float(np.std(r_all) + 1e-6)
    }

def choose_max_seq(lengths, override=None):
    if override is not None:
        return int(override)
    p95 = int(np.percentile(lengths, 95))
    p95 = max(8, ((p95 + 7) // 8) * 8)  # round up to multiple of 8
    return p95

# Run
sessions, labels, lengths = collect_sessions(DATA_ROOT)
print(f"Found {len(sessions)} sessions. lengths: min={min(lengths)}, max={max(lengths)}, mean={np.mean(lengths):.1f}")
stats = compute_stats(sessions)
print("Computed normalization stats:", stats)
max_seq = choose_max_seq(lengths, override=MAX_SEQ_OVERRIDE)
print("Chosen MAX_SEQ_LEN =", max_seq)

# Save stats for later use on device
with open(STATS_OUT, "w") as fh:
    json.dump(stats, fh, indent=2)
print("Saved normalization stats to", STATS_OUT)

Found 42 sessions. lengths: min=36, max=39, mean=37.0
Computed normalization stats: {'thermal_mean': 15.55591869354248, 'thermal_std': 10.219653129577637, 'radar_mean': 78054.875, 'radar_std': 514466.59375}
Chosen MAX_SEQ_LEN = 40
Saved normalization stats to norm_stats.json


In [6]:
# Cell 5: Pad/truncate sessions and build tf.data datasets
def pad_truncate_session(sess, stats, max_len):
    L = len(sess)
    therm_p = np.zeros((max_len, THERM_H, THERM_W, THERM_C), dtype=np.float32)
    radar_p = np.zeros((max_len, RADAR_DIM), dtype=np.float32)
    t_mean, t_std = stats["thermal_mean"], stats["thermal_std"]
    r_mean, r_std = stats["radar_mean"], stats["radar_std"]
    for i in range(min(L, max_len)):
        t, r = sess[i]
        therm_p[i] = (t - t_mean) / t_std
        radar_p[i] = (r - r_mean) / r_std
    if L < max_len and L > 0:
        therm_p[L:] = therm_p[L-1]
        radar_p[L:] = radar_p[L-1]
    return therm_p, radar_p, min(L, max_len)

def build_tf_dataset(sessions, labels, stats, max_len, batch, shuffle=True):
    X1=[]; X2=[]; L=[]
    for sess in sessions:
        t_p, r_p, length = pad_truncate_session(sess, stats, max_len)
        X1.append(t_p); X2.append(r_p); L.append(length)
    X1 = np.array(X1, dtype=np.float32)
    X2 = np.array(X2, dtype=np.float32)
    L  = np.array(L, dtype=np.int32)
    Y  = np.array(labels, dtype=np.int32)
    ds = tf.data.Dataset.from_tensor_slices(((X1, X2, L), Y))
    if shuffle:
        ds = ds.shuffle(buffer_size=min(2048, len(Y)))
    ds = ds.batch(batch).prefetch(tf.data.AUTOTUNE)
    return ds, X1, X2, L

# train/val split
N = len(sessions)
idx = np.random.permutation(N)
split = int(0.8 * N)
train_idx, val_idx = idx[:split], idx[split:]
train_s = [sessions[i] for i in train_idx]; train_y = [labels[i] for i in train_idx]
val_s   = [sessions[i] for i in val_idx];   val_y   = [labels[i] for i in val_idx]

train_ds, X1_train, X2_train, L_train = build_tf_dataset(train_s, train_y, stats, max_seq, BATCH, shuffle=True)
val_ds,   X1_val,   X2_val,   L_val   = build_tf_dataset(val_s, val_y, stats, max_seq, BATCH, shuffle=False)

print("Built datasets. Train samples:", X1_train.shape[0], "Val samples:", X1_val.shape[0])

Built datasets. Train samples: 33 Val samples: 9


In [8]:
# Cell 6 (REPLACEMENT): Keras model definition with MaskedAvgPool layer

import tensorflow as tf

class MaskedAvgPool(tf.keras.layers.Layer):
    """
    Inputs:
      - fused: (B, seq, feat)
      - lengths: (B,) int32
    Output:
      - avg pooled (B, feat) where pooling ignores padding frames (frames >= length)
    """
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, inputs):
        fused, lengths = inputs  # fused: (B, seq, feat), lengths: (B,)
        seq_len = tf.shape(fused)[1]                       # dynamic seq length
        # build mask: shape (1, seq) -> broadcast to (B, seq)
        idx = tf.range(seq_len)[None, :]                   # (1, seq)
        mask = tf.cast(tf.less(idx, lengths[:, None]), tf.float32)  # (B, seq)
        mask = mask[..., None]                             # (B, seq, 1)
        fused_masked = fused * mask                        # zero out padded frames
        summed = tf.reduce_sum(fused_masked, axis=1)       # (B, feat)
        lengths_f = tf.cast(tf.maximum(lengths, 1), tf.float32)[:, None]  # (B,1)
        avg = summed / lengths_f                           # (B, feat)
        return avg

    def get_config(self):
        return super().get_config()

def make_keras_model(max_len):
    t_in = tf.keras.Input(shape=(max_len, THERM_H, THERM_W, THERM_C), name="thermal")
    r_in = tf.keras.Input(shape=(max_len, RADAR_DIM), name="radar")
    length = tf.keras.Input(shape=(), dtype=tf.int32, name="length")

    td = tf.keras.layers.TimeDistributed
    x = td(tf.keras.layers.Conv2D(8, 3, padding="same", activation="relu"))(t_in)
    x = td(tf.keras.layers.MaxPool2D(2))(x)   # -> 4x4
    x = td(tf.keras.layers.Conv2D(12, 3, padding="same", activation="relu"))(x)
    x = td(tf.keras.layers.Flatten())(x)
    frame_feat = td(tf.keras.layers.Dense(32, activation="relu"))(x)  # (B, seq, 32)

    radar_feat = td(tf.keras.layers.Dense(16, activation="relu"))(r_in)  # (B, seq, 16)
    fused = tf.keras.layers.Concatenate(axis=-1)([frame_feat, radar_feat])  # (B, seq, 48)

    # masked average pooling via custom layer
    avg = MaskedAvgPool()([fused, length])  # returns (B, feat)

    x = tf.keras.layers.Dense(24, activation="relu")(avg)
    out = tf.keras.layers.Dense(2, activation="softmax", dtype="float32")(x)
    model = tf.keras.Model(inputs=[t_in, r_in, length], outputs=out)
    return model

# create model
model = make_keras_model(max_seq)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ thermal             │ (None, 40, 8, 8,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_6  │ (None, 40, 8, 8,  │        224 │ thermal[0][0]     │
│ (TimeDistributed)   │ 8)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_7  │ (None, 40, 4, 4,  │          0 │ time_distributed… │
│ (TimeDistributed)   │ 8)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_8  │ (None, 40, 4, 4,  │        876 │ time_distributed… │
│ (TimeDistributed)   │ 12)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_9  │ (None, 40, 192)   │          0 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ radar (InputLayer)  │ (None, 40, 12)    │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_10 │ (None, 40, 32)    │      6,176 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_11 │ (None, 40, 16)    │        208 │ radar[0][0]       │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 40, 48)    │          0 │ time_distributed… │
│ (Concatenate)       │                   │            │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ length (InputLayer) │ (None)            │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masked_avg_pool     │ (None, 48)        │          0 │ concatenate_1[0]… │
│ (MaskedAvgPool)     │                   │            │ length[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 24)        │      1,176 │ masked_avg_pool[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 2)         │         50 │ dense_4[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 8,710 (34.02 KB)

 Trainable params: 8,710 (34.02 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Cell 7: Train
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.5),
    tf.keras.callbacks.EarlyStopping(patience=6, restore_best_weights=True)
]
history = model.fit(train_ds, epochs=EPOCHS, validation_data=val_ds, callbacks=callbacks)
print("Saved Keras model to saved_model_tf")

Epoch 1/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 267ms/step - accuracy: 0.6061 - loss: 0.6641 - val_accuracy: 0.4444 - val_loss: 0.7187 - learning_rate: 0.0010
Epoch 2/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 267ms/step - accuracy: 0.6061 - loss: 0.6641 - val_accuracy: 0.4444 - val_loss: 0.7187 - learning_rate: 0.0010
Epoch 2/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.6364 - loss: 0.6497 - val_accuracy: 0.4444 - val_loss: 0.7061 - learning_rate: 0.0010
Epoch 3/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.6364 - loss: 0.6497 - val_accuracy: 0.4444 - val_loss: 0.7061 - learning_rate: 0.0010
Epoch 3/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6364 - loss: 0.6486 - val_accuracy: 0.5556 - val_loss: 0.6904 - learning_rate: 0.0010
Epoch 4/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6364 - loss: 0.6486 - val_accuracy: 0.5556 - val_loss: 0.6904 - learning_rate: 0.0010
Epoch 4/15
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6970 - loss: 0.6507 - val_accuracy: 0.5556 -

ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=saved_model_tf.